In [206]:
using LowLevelFEM

In [207]:
gmsh.initialize()

In [208]:
gmsh.open("cube1.geo")

Info    : Reading 'cube1.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.00326694s, CPU 0.001268s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.000177811s, CPU 0.000135s)
Info    : Meshing 3D.

In [209]:
mat = material("cube")
prob = Problem([mat])

Problem("cube1", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("cube", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 8)

In [210]:
supp = displacementConstraint("rear", ux=0, uy=0, uz=1)
load1 = load("front", fz=100000)

("front", 0, 0, 100000)

In [211]:
K = stiffnessMatrix(prob)

24×24 SparseArrays.SparseMatrixCSC{Float64, Int64} with 576 stored entries:
⎡⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎤
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎣⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎦

In [212]:
f = loadVector(prob, [load1])

VectorField

24×1 Matrix{Float64}:
     0.0
     0.0
 25000.0
     0.0
     0.0
     0.0
     0.0
     0.0
 25000.0
     0.0
     0.0
     0.0
     0.0
     0.0
 25000.0
     0.0
     0.0
     0.0
     0.0
     0.0
 25000.0
     0.0
     0.0
     0.0

In [213]:
free = freeDoFs(prob, [supp])
fixed = constrainedDoFs(prob, [supp])

12-element Vector{Any}:
  4
 10
 16
 22
  5
 11
 17
 23
  6
 12
 18
 24

In [214]:
u = copy(f)

VectorField

24×1 Matrix{Float64}:
     0.0
     0.0
 25000.0
     0.0
     0.0
     0.0
     0.0
     0.0
 25000.0
     0.0
     0.0
     0.0
     0.0
     0.0
 25000.0
     0.0
     0.0
     0.0
     0.0
     0.0
 25000.0
     0.0
     0.0
     0.0

In [215]:
fill!(u.a, 0.0)

24×1 Matrix{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [216]:
applyBoundaryConditions!(u, [supp])
u

VectorField

24×1 Matrix{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0

In [217]:
f_kin = K.A[:, fixed] * u.a[fixed]

24-element Vector{Float64}:
  28846.15384615385
  28846.15384615385
 -67307.6923076923
  28846.153846153848
  28846.15384615385
  67307.69230769231
  28846.153846153844
 -28846.153846153844
 -67307.69230769231
  28846.153846153848
 -28846.15384615385
  67307.69230769231
 -28846.153846153848
  28846.15384615385
 -67307.69230769231
 -28846.153846153844
  28846.153846153848
  67307.6923076923
 -28846.153846153855
 -28846.15384615385
 -67307.69230769231
 -28846.153846153844
 -28846.15384615385
  67307.69230769231

In [218]:
u.a[free] = K.A[free, free] \ (f.a[free] - f_kin[free])

12-element Vector{Float64}:
  0.0975000000000001
  0.0974999999999998
  1.4549999999999998
  0.09750000000000066
 -0.09750000000000071
  1.4550000000000012
 -0.09749999999999968
  0.0974999999999996
  1.455
 -0.09749999999999971
 -0.09750000000000016
  1.455

In [219]:
u

VectorField

24×1 Matrix{Float64}:
  0.0975000000000001
  0.0974999999999998
  1.4549999999999998
  0.0
  0.0
  1.0
  0.09750000000000066
 -0.09750000000000071
  1.4550000000000012
  0.0
  0.0
  1.0
 -0.09749999999999968
  0.0974999999999996
  1.455
  0.0
  0.0
  1.0
 -0.09749999999999971
 -0.09750000000000016
  1.455
  0.0
  0.0
  1.0

In [220]:
showDoFResults(u)

0

In [221]:
showDoFResults(f)

1

In [222]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [223]:
gmsh.finalize()

In [224]:
for col in 1:size(K, 2)
    colrange = K.colptr[col]:(K.colptr[col+1]-1)
    for rowidx in colrange
        row = K.rowval[rowidx]
        if fixed[row]
            K.nzval[rowidx] = 0.0
        elseif fixed[col]
            K.nzval[rowidx] = 0.0
        end
    end
end
for dof in fixed_dofs
    K[dof, dof] = 1.0  # ha létezett korábban
end


LoadError: MethodError: no method matching size(::LowLevelFEM.SystemMatrix, ::Int64)
The function `size` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  size([91m::BitVector[39m, ::Integer)
[0m[90m   @[39m [90mBase[39m [90m[4mbitarray.jl:107[24m[39m
[0m  size([91m::Type{LinearAlgebra.Diagonal{T, StaticArraysCore.SVector{N, T}} where T}[39m, ::Int64) where N
[0m[90m   @[39m [33mStaticArrays[39m [90m~/.julia/packages/StaticArrays/DsPgf/src/[39m[90m[4mSDiagonal.jl:19[24m[39m
[0m  size([91m::Type{LinearAlgebra.Diagonal{T, StaticArraysCore.SVector{N, T}}}[39m, ::Int64) where {N, T}
[0m[90m   @[39m [33mStaticArrays[39m [90m~/.julia/packages/StaticArrays/DsPgf/src/[39m[90m[4mSDiagonal.jl:20[24m[39m
[0m  ...
